## ComplaintsDataset is our initial full dataset. Problem_Data is the ComplaintsDataset but after preprocessing and contains all the complaints classified in 5 categories. Problem_Data_512 contains the same minus the complaints with length greater than 512. Load dataset drop empty rows and duplicates. We don't need the labels in this notebook, but watch out when we group them together, the result of grouped categories will be different depending on the preprocessing

In [ ]:
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # display the whole dataframe rows
# pd.set_option('display.max_colwidth', None)

# # display all dataframe rows
# pd.set_option('display.max_rows', None)

datapath = 'ComplaintsDataset.csv'

data = pd.read_csv(datapath)

data = data[['Λεπτομέρειες','Υποενότητα']].rename(columns={'Λεπτομέρειες':'text','Υποενότητα':'label'})

# drop all rows with NaN values
data.dropna(subset=['text'], inplace=True)
print('NaN values in text column: ',data['text'].isna().sum())

data = data.drop(['label'], axis=1)

# # Filter and print duplicate rows
# print(data['text'][data['text'].duplicated(keep=False)])
data = data.drop_duplicates(subset=['text'])
print("Number of duplicates:", data['text'].duplicated().sum())

# make column with original complaints
data = data.assign(init_text=data['text'])

# data=data.dropna()
data = data.reset_index(drop = True)

NaN values in text column:  0
Number of duplicates: 0


In [ ]:
import re
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# #frequency of each label
# label_counts = data['label'].value_counts()
# # Merge all low-frequency classes into the class "Άλλο"
# low_freq_labels = label_counts[label_counts < 380].index.tolist()
# data.loc[data['label'].isin(low_freq_labels), 'label'] = 'Άλλο'
# # Save labels in a list
# label_list = data["label"].unique()
# # Map the labels to integers
# label_ids = {label: i for i, label in enumerate(label_list)}
# #frequency of each label
# label_counts = data['label'].value_counts()
# print(label_counts)
# # # Replace label names with their corresponding values
# # data['label'] = [label_ids[label] for label in data['label']]

# strip accents and lowercase function
def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()

data['text'] = data['text'].apply(strip_accents_and_lowercase)

# remove stop words
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('greek'))

# Define a function to remove stopwords, special symbols, and tokenize the text
additional_stopwords = ["απο","από", "μου", "στις", "ειναι","είναι", "καθώς","μας","ότι","ήταν","αλλά","ενώ","τους","αυτό","εχει","πρέπει","ή","γιατί", "όπως","ούτε", "όταν", "χωρίς", "ένα", "αφού", "έχει", "είχε"]
def remove_stopwords(text):
    # Remove special symbols using regex
    text = re.sub(r"[^\w\s]", "", text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove links and URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Tokenize the text
    # words = word_tokenize(text)
    words = text.split()
    # words = [word for word in words if word.lower() not in stop_words]
    words = [word for word in words if word.lower() not in stop_words and word.lower() not in additional_stopwords]
    return " ".join(words)

data['text'] = data['text'].apply(remove_stopwords)

data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,text,init_text
0,καλημερα σας κατοχος της υπαριθμον ανωνυμη καρ...,Καλημερα σας ειμαι κατοχος της υπαριθμον 300...
1,αγαπητοι κυριοι καλησπερα σας σημερα ωρα απογε...,Αγαπητοί κύριοι καλησπέρα σας σήμερα 31/1 2022...
2,στης μπηκα λεωφορειο παω τερμα δυο στασεις προ...,στης 19.01 μπήκα στο λεωφορείο 022 για να παω ...
3,σταθμος τραμ γηπεδο καραισκακη τηλεφωνο εκτακτ...,Σταθμός Τραμ Γήπεδο Καραϊσκάκη. Το τηλέφωνο έκ...
4,σταθμος κεραμεικος γραμμη επιχειρησα ανανεωσω ...,"Σταθμός Κεραμεικός, γραμμή 3 . Επιχείρησα να α..."
...,...,...
6637,καλημερα σας παρακαλω ηθελα ενημερωθω ακυρωση ...,Καλημέρα σας!\n\nΠαρακαλώ θα ήθελα να ενημερωθ...
6638,καλημερα σας καλο μηνα λεωφορειο τερμα της ζωο...,Καλημέρα σας και καλό μηνα. Το λεωφορείο 730 α...
6639,καλημερα σας απαραδεκτος οδηγος καθως κοπελα η...,"Καλημέρα σας, \nΑπαράδεκτος οδηγός, καθώς η κο..."
6640,δρομολογιο ενημερωστε γιατι περασε,Το δρομολόγιο των 7 και 20 ενημερώστε με γιατί...


## Counts texts with more than 512 tokens and then discards them

In [ ]:
# !pip install transformers
# from transformers import AutoTokenizer
# # Load greek BERT tokenizer
# tokenizer = AutoTokenizer.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')
# #tokenize texts and find number of texts with more than 512 tokens
# LongTextsCount = 0
# for text in data['text']:
#     tokenized = tokenizer.encode(text, truncation=False,return_tensors='pt', add_special_tokens=True)
#     if tokenized.size()[1] > 512:
#         LongTextsCount += 1
# print("\n\nNumber of texts with more than 512 tokens: ", LongTextsCount)
# # Drop long texts since there's too few of them
# LongTextIdx = []
# for i, text in enumerate(data['text']):
#     tokenized = tokenizer.encode(text, truncation=False, return_tensors='pt', add_special_tokens=True)
#     if tokenized.size()[1] > 512:
#         LongTextIdx.append(i)
# data_rem = data.drop(index=LongTextIdx).reset_index(drop=True)
# data_rem

## make a list of words and search for them, or for words that partially match with them, inside the texts - keep only the sentences that contain such words

In [ ]:
import nltk
import string
from nltk.tokenize import sent_tokenize

# Download the sentence tokenizer
nltk.download('punkt')

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# words that we are looking for - 2 categories
strings_law = ['δικαστ', 'νομικ', 'αγωγ', 'αποζημι', 'αποζημί', 'δικαιοσ', 'αδικημ', 'αδίκημ', 'δίκη', 'μηνυσ', 'μήνυσ']
strings_hurt = ['τραυμ', 'διαστρεμ', 'διάστρεμ', 'κακωση', 'κάκωση', 'κάταγμα', 'βάρεσ', 'βαρά', 'βαραει', 'βαραω', 'βαρεσ',
                'καταγμα', 'χτυπ', 'χτύπ', 'νοσοκομ', 'επίθεσ', 'επιθεσ', 'επιτεθ', 'επιθετ', 'επιτέθ']

# keep only texts that contain keywords but beware that words
# can only start with our strings to keep and not end with them
# for example "ξαναχτυπάω" won't match
pattern = r'(?<=[\s\W])(?:' + '|'.join(map(re.escape, strings_law + strings_hurt)) + r')'
filtered_df = data[data['text'].str.contains(pattern, regex=True)]

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# our texts might have dots or other symbols that are immediately followed by string with
# no space. the below goes through the text and adds a space where such a thing occurs
def add_space_after_symbol(df, column_name):
    new_df = df.copy()
    pattern = r'[-.,()…](?=\S)'
    # Function to add a space after the symbol
    def add_space(match):
        return match.group(0) + ' '
    new_df[column_name] = new_df[column_name].apply(lambda x: re.sub(pattern, add_space, x))
    return new_df

filtered_df = add_space_after_symbol(filtered_df, "text")

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# # def find_occurrence(text, search_string, k):
# #     pattern = r"(?<!\w){}(?!\w)".format(re.escape(search_string))
# #     matches = re.findall(pattern, text)
# #     print("matches ", matches)
# #     if len(matches) < k:
# #         return None
# #     start_index = text.find(matches[k - 1])
# #     end_index = start_index + len(matches[k - 1]) - 1
# #     return (start_index, end_index)
# # find the start and end position of a string in a particular
# # text take into account that some times the same word
# # can be counted in the matches more than one time
# def find_occurrence(search_string, text, k):
#     start_index = text.find(search_string)
#     while start_index >= 0 and k > 1:
#         start_index = text.find(search_string, start_index + 1)
#         k -= 1
#     if start_index < 0:
#         return None
#     end_index = start_index + len(search_string) - 1
#     return (start_index, end_index)
# def print_substring(text, start, end):
#     substring = text[start:end+1]
#     print(substring)
# # might not be used. replaces specific characters with space
# def replace_characters_with_space(text):
#     characters = ["»", "…"]
#     for char in characters:
#         text = text.replace(char, ' ')
#     return text
# # check if a token is space or Punctuation
# def is_space_or_punctuation(char):
#     return char.isspace() or char in string.punctuation or char=="»" or char=="…"

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# return the position number of the beginning of the immediate identical match of string in a text
def find_next_identical_match(text, my_string):
    pattern = r'\b{}\b'.format(re.escape(my_string))
    match = re.search(pattern, text)
    if match:
        return match.start()
    else:
        return -1

# returns the beginning and end position numbers of the k-th ocurrence of a string in a text
def ret_beginning_and_end_of_kth_occurence(my_txt, search_string, k):
    # my_txt = replace_characters_with_space(my_txt)
    start = find_next_identical_match(my_txt, search_string)
    while start >= 0 and k > 1:
        start = start + len(search_string) + find_next_identical_match(my_txt[start + len(search_string):], search_string)
        k -= 1
    return start, start + len(search_string)-1

# takes two strings and returns True if they are same. beware that if str1 is
# same with str2 but also contains punctuation, it will also be considered same
def compare_strings_forget_punct(str1, str2):
    # Concatenate the standard punctuation and additional punctuations
    additional_punctuations = "[]{}()!@#$%^&*+=_-|\/?.,;:…"
    all_punctuations = string.punctuation + additional_punctuations
    translator = str.maketrans("", "", all_punctuations)
    str1 = str1.translate(translator)
    str2 = str2.translate(translator)
    return str1 == str2

# takes a list of words and a word and returns the index
# of the word in the list using the compare_strings_forget_punct
def get_word_index_from_list(list_words, word):
  counter = 0
  while counter <= len(list_words):
      if compare_strings_forget_punct(list_words[counter], word):
          return counter
      else:
          counter += 1

# given a text, a word and a number k, return the word index of the k-th word occurence in the text
def find_word_serial_number(text, word, k):
    # the number of previous same words
    offset_words = 0
    words = text.split()
    while k>0:
      try:
          # serial_number = words.index(word) + 1
          serial_number = get_word_index_from_list(words, word)
          k -= 1
          if k == 0:
            return serial_number + offset_words
          else:
            offset_words += serial_number + 1
            # keep only the remaining words in the list discard the previous
            # since we have already counted them
            words = words[serial_number+1:]
      except ValueError:
          return -1

# starting from the dataframe texts we keep, go further and only keep sentences
# which contain strings to keep beware though that we will discard sentences in
# which we find words like 'χτυπάω', 'χτύπησα' e.g. and they refer to tickets
def filter_sentences_discard_tickets(text, keywords):
    related_to_tickets = ['χτυπ', 'χτύπ']
    ticket_strings = ['εισητ', 'εισιτ', 'εισειτ', 'καρτ', 'κάρτ']
    filtered_sentences = []
    sentences = sent_tokenize(text)

    for sentence in sentences:
        for keyword in keywords:
            # now check if those sentences contain texts like 'χτυπαω εισητηριο'
            if keyword in related_to_tickets:
                pattern = r"\b{}\w*\b".format(keyword)
                cond = re.search(pattern, sentence, flags=re.IGNORECASE)
                keep_sen = True
                if cond:
                  matches = re.findall(pattern, sentence)
                  for word_ind in range(len(matches)):

                      # # find position of first and last tokens of word in sentence
                      # word_start, word_end = ret_beginning_and_end_of_kth_occurence(sentence, matches[word_ind], word_ind+1)

                      # number of same words in the matches list before the current one
                      count_found = 1
                      previous_words = matches[:word_ind]
                      for i in previous_words:
                          if compare_strings_forget_punct(i, matches[word_ind]):
                              count_found += 1

                      # get word index in text
                      word_in_text = find_word_serial_number(sentence, matches[word_ind], count_found)

                      # remove punctuation
                      sentence_no_pun = sentence.translate(str.maketrans('', '', string.punctuation))
                      sentence_words = nltk.word_tokenize(sentence_no_pun)

                      # now we will check if in a sequence of words like w1 w2 "χτυπάω" w3 w4, a word like
                      # "εισητηριο" is found. we will take the 2 previous and two following words if they exist
                      # or else we take less words in the sequence
                      min_word_ind = max(0, word_in_text-3)
                      max_word_ind = min(word_in_text+4, len(sentence_words))

                      before_and_after_words = sentence_words[min_word_ind:max_word_ind]

                      for word_check in before_and_after_words:
                          for ticket_string in ticket_strings:
                              pattern = r"\b{}\w*\b".format(ticket_string)
                              if re.search(pattern, word_check, flags=re.IGNORECASE):
                                  keep_sen = False
                                  break
                  if keep_sen:
                    filtered_sentences.append(sentence)
            else:
                pattern = r"\b{}\w*\b".format(keyword)
                cond = re.search(pattern, sentence, flags=re.IGNORECASE)
                if cond:
                    filtered_sentences.append(sentence)
                    break
    return ' '.join(filtered_sentences)

# starting from the dataframe texts we keep, only keep sentences which contain strings to keep
def filter_sentences(text, keywords):
    sentences = sent_tokenize(text)
    filtered_sentences = []
    for sentence in sentences:
        for keyword in keywords:
            pattern = r"\b{}\w*\b".format(keyword)
            if re.search(pattern, sentence, flags=re.IGNORECASE):
                filtered_sentences.append(sentence)
                break
    return ' '.join(filtered_sentences)

# create and return a list of the words from the dataframe texts which
# match with strings to keep. don't keep words that end with our keywords
def find_matching_strings(text, mylist):
    pattern = r'(?<=[\s\W])(?:' + '|'.join(map(re.escape, mylist)) + r')(?:\w+)?'
    matches = re.findall(pattern, text)
    return matches

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np

filtered_df = filtered_df.copy()

# keep only sentences with keywords
# filtered_df['text'] = filtered_df['text'].apply(lambda x: filter_sentences(x, strings_law + strings_hurt))
filtered_df['text'] = filtered_df['text'].apply(lambda x: filter_sentences_discard_tickets(x, strings_law + strings_hurt))

# Add column that contains a list of words found in dataframe text that match with the strings to keep
filtered_df['Matches'] = filtered_df['text'].apply(lambda x: find_matching_strings(x, strings_law + strings_hurt))

# drop duplicates
filtered_df=filtered_df.dropna()
filtered_df = filtered_df.loc[filtered_df['text'] != " "]

# Filter the dataframe to keep only rows that contain characters
filtered_df['Contains_Characters'] = filtered_df['text'].apply(lambda x: np.any([c.isalpha() for c in str(x)]))
filtered_df = filtered_df[filtered_df['Contains_Characters']]

filtered_df = filtered_df.drop(['Contains_Characters'], axis=1)

# print(filtered_df.shape)
# filtered_df[:4]
filtered_df
# filtered_df.loc[3027]

,text,init_text,Matches
1,αγαπητοι κυριοι καλησπερα σας σημερα ωρα απογε...,Αγαπητοί κύριοι καλησπέρα σας σήμερα 31/1 2022...,[αποζημιωσετε]
16,παρακαλω πολυ σημερα κυριακη ωρα μμ λεωφορειο ...,"Παρακαλώ πολύ, σήμερα, Κυριακή, 30 /01 / 2022 ...",[μηνυση]
43,aπο χθες πρωι υπαρχει ξεκιναει κανενα τρολει α...,Aπό χθες το πρωι δεν υπαρχει και δεν ξεκιναει ...,[αποζημιωσει]
62,πρεπει κινηθω εργασια λαικο νοσοκομειο,Πρέπει να μετά κινηθώ στη εργασία μου στο Λαϊκ...,[νοσοκομειο]
80,πρωι σαββατου εχοντας σχολασει απ τη δουλεια ε...,"Το πρωί του Σαββάτου, έχοντας σχολασει απ' τη ...",[χτυπαω]
...,...,...,...
6593,καλημερα ονομαζομαι δαναη ορφανου τσωτα επικοι...,"Καλημέρα, ονομάζομαι Δανάη Ορφανού Τσώτα. Επικ...","[χτυπησα, τραυματιστει]"
6618,γινει ρε οασα οδηγους σου καπνιζουν μεσα στα ο...,Τι θα γίνει ρε ΟΑΣΑ με τους οδηγούς σου που κα...,[μηνυση]
6622,εσταζε ολη οροφη νερα της βροχης διαδρομο τρεχ...,Έσταζε όλη η οροφή από τα νερά της βροχής στον...,[χτυπησεις]
6623,απαραδεκτος οδηγος υπ αρθμον χεκ διασταυρωση δ...,ΑΠΑΡΑΔΕΚΤΟΣ ΟΔΗΓΟΣ ΤΟΥ ΥΠ ΑΡΘΜΟΝ ΧΕΚ 6168 ΣΤΗΝ...,"[δικαιοσυνη, νομικες]"


## conversion to JSON and save a json file of dictionaries

In [ ]:
import pandas as pd
import json

# gets a list of strings and a string and returns the number of occurences of the string in the list
def count_occurrences(string_list, target_string):
    count = 0
    for string in string_list:
        if string == target_string:
            count += 1
    return count

# create a list of the json data as should be for spacy
def dataframe_to_json(df):
    json_list = []
    for index, row in df.iterrows():
        json_dict = {'content': row['text']}

        list_of_labels = []
        # some words exist in a list twice
        list_of_checked_words = []
        # the words in the texts that match with our keywords
        for word in row['Matches']:

          counted_already = count_occurrences(list_of_checked_words, word)

          # C H E C K
          start, end = ret_beginning_and_end_of_kth_occurence(row['text'], word, counted_already+1)

          list_of_checked_words.append(word)

          # assign the label as Hurt and if find that it is
          # Law, change it
          label_dict = {"label": ["B-HUR"]}

          for keyword in strings_law:

            pattern = r"\b{}\w*\b".format(keyword)

            if re.search(pattern, word, flags=re.IGNORECASE):
              label_dict = {"label": ["B-LAW"]}
              break

          label_dict["points"] = [
              {
              'text': str(word),
              'start': start,
               'end': end
              }
          ]

          list_of_labels.append(label_dict)

        json_dict["annotation"] = list_of_labels
        json_list.append(json.dumps(json_dict))
    return json_list

json_list = dataframe_to_json(filtered_df)

for i in json_list[:2]:
  json_object = json.loads(i)
  json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False)
  print(json_formatted_str)

# save the json file
with open('/content/drive/My Drive/arximhdhs/ner/my_json_data.json', "w", encoding="utf-8") as file:
  for inner_dict in json_list:
    json_object = json.loads(inner_dict)
    json_data = json.dumps(json_object, ensure_ascii=False)
    file.write(json_data)
    file.write('\n')

{
  "content": "αγαπητοι κυριοι καλησπερα σας σημερα ωρα απογευμα κορη ετων βρισκοταν τερμα σαρωνιδα προορισμο λαγονησι δρομολογιο εκτελεστηκε θεση βρισκοταν υπ αριθμον χεη λεωφορειο σταθμευμενο εμπροσθεν της τασης πραγματοποιωντας λεγομενα οδηγου διαλειμμα οδηγος αγενεστατος οταν ρωτηθηκε κορη ωρα φυγει αρκεστηκε πει λεπτα εχοντας κλειστες πορτες αφηνοντας κοσμο κρυο οδηγος βρισκοταν δυο ατομα εντος λεωφορειου μιλαγε οδηγος ταξι πραγματοποιησα δρομολογιο βαρκιζα σαρωνιδα χωρις δρομο συναντησω αρκετα λεπτα αργοτερα υπ αριθμον χεη πραγματοποιησε κανονικα δρομολογιο ευγενεστατο οδηγο αγαπητοι κυριοι οασα της οσοι πολιτικη σας αφηνετε παιδια μεσα κρυο ακομα λεωφορειο κανει διαλειμμα κορη εχασε μαθημα λαγονησι εγω ετρεξα προλαβω αποζημιωσετε εχετε σκοπο φτιαξετε τη γραμμη μεγαλη αγανακτηση μαγκας κωνσταντινος διαθεση σας φωτογραφια λεωφορειο κανει διαλειμμα μπροστα σταση τερματος κατι τη γνωμη απαραδεκτη",
  "annotation": [
    {
      "label": [
        "B-LAW"
      ],
      "points": [


## convert the above data into format needed by spaCy and save in a pickle file

In [ ]:
import logging
import argparse
import sys
import os
import json
import pickle

def conversion_function(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        # with open(input_file, 'r', encoding="utf-8") as f:
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            # data = json.dumps(data, ensure_ascii=False)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]
                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))
            training_data.append((text, {"entities" : entities}))
        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None

conversion_function('/content/drive/My Drive/arximhdhs/ner/my_json_data.json', '/content/drive/My Drive/arximhdhs/ner/my_json_pkl.pkl')

## now our data is ready and we can train a NER model

In [ ]:
# pip install -U pip setuptools wheel
!pip install -U spacy
# pip install -U 'spacy[cuda-autodetect]'

import spacy.cli
print()
print(spacy.__version__)
print()
spacy.cli.download("el_core_news_sm")
# python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2

3.5.3

✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


## Load the model, or create an empty model using spacy.blank with the ID of desired language. If a blank model is being used, we have to add the entity recognizer to the pipeline. If an existing model is being used, we have to disable all other pipeline components during training using nlp.disable_pipes. This way, only the entity recognizer gets trained. Add the new entity label to the entity recognizer using the add_label method.

In [ ]:
# Training additional entity types using spaCy
from spacy.util import minibatch, compounding
import random
from pathlib import Path
import spacy
from spacy.training.example import Example
# from sklearn.model_selection import train_test_split


with open ('/content/drive/My Drive/arximhdhs/ner/my_json_pkl.pkl', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)


# Test the trained model
def test_trained_model(df, column, num, my_nlp):
  print()
  test_text = df[column].iloc[num]
  doc = my_nlp(test_text)
  print("Entities in '%s'" % test_text)
  for ent in doc.ents:
      print(ent.label_, ent.text)


def train_my_model(model=None, new_model_name='our_NER_model', output_dir=None, n_iter=26):
    """Setting up the pipeline and entity recognizer, and training the new entity."""
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)

    batch_size = 128
    drop_out = 0.15
    learning_rate = 0.001
    regularization = 1e-6
    grad_clip_value = 1.0

    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe('ner')
        print("ner ", ner)

    # # Specify the new entity labels which you want to add here
    # LABEL = ["B-HUR", "B-LAW"]
    # # Add new entity labels to entity recognizer
    # for i in LABEL:
    #     ner.add_label(i)
    # Add your custom labels to the NER component
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER

        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = ner.create_optimizer()

        optimizer.learn_rate = learning_rate
        optimizer.L2 = regularization
        optimizer.grad_clip = grad_clip_value

        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}

            for batch in minibatch(TRAIN_DATA, size=batch_size):
              for text, annotations in batch:
                  doc = nlp.make_doc(text)

                  example = Example.from_dict(doc, annotations)
                  # the drop parameter controls the dropout rate, which helps prevent overfitting
                  # nlp.update([example], losses=losses, drop=drop_out)  # Update the NER model
                  nlp.update([example], losses=losses, drop=drop_out, sgd=optimizer)  # Update the NER model

            print(f"Iteration {itn+1}: Train Losses {losses}")

    test_trained_model(filtered_df, 'text', 145, nlp)
    test_trained_model(filtered_df, 'text', 45, nlp)
    test_trained_model(filtered_df, 'text', 67, nlp)

    print()
    print()

    # Save model
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # Test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        test_trained_model(filtered_df, 'text', 34, nlp2)

In [ ]:
train_my_model('el_core_news_sm', 'our_NER_model', '/content/drive/My Drive/arximhdhs/ner/new_our_NER_model', 26)

Loaded model 'el_core_news_sm'
ner  <spacy.pipeline.ner.EntityRecognizer object at 0x7f3d820f68f0>
Iteration 1: Train Losses {'ner': 704.2147681896428}
Iteration 2: Train Losses {'ner': 160.55492481198218}
Iteration 3: Train Losses {'ner': 75.33999263396632}
Iteration 4: Train Losses {'ner': 46.08789793582352}
Iteration 5: Train Losses {'ner': 25.716718812045233}
Iteration 6: Train Losses {'ner': 29.394540499555422}
Iteration 7: Train Losses {'ner': 44.536981704031035}
Iteration 8: Train Losses {'ner': 11.476509322398973}
Iteration 9: Train Losses {'ner': 32.19116715104209}
Iteration 10: Train Losses {'ner': 41.088112039414526}
Iteration 11: Train Losses {'ner': 34.41297370137192}
Iteration 12: Train Losses {'ner': 16.890464505195197}
Iteration 13: Train Losses {'ner': 11.849082397873499}
Iteration 14: Train Losses {'ner': 3.420062469573205}
Iteration 15: Train Losses {'ner': 10.345500249155172}
Iteration 16: Train Losses {'ner': 21.908732797937468}
Iteration 17: Train Losses {'ner': 1

In [ ]:
# Test the saved model
output_dir = '/content/drive/My Drive/arximhdhs/ner/new_our_NER_model'
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
test_trained_model(filtered_df, 'text', 34, nlp2)
test_trained_model(filtered_df, 'text', 43, nlp2)
test_trained_model(filtered_df, 'text', 167, nlp2)

Loading from /content/drive/My Drive/arximhdhs/ner/new_our_NER_model

Entities in 'καλησπερα ηθελα σας καταγγειλω διαδρομη λεωφορειο αφετηρια αττικο νοσοκομειο προς νικαια συγκεκριμενα δρομολογιο αττικο σχεδον καθημερνη βαση υπαρξη της προστατευτικης μασκας ανυπαρκτη οδηγος παρατηρηση απαντησε εκεινος μπορει κανει τιποτα επειδη κατασταση σχεδον καθημερινοτητα ειδικα αυτες ωρες αναγκαζομαστε εργαζομενοι νοσοκομειου επισκεπτες νοσοκομειου μπορουμε παιρνουμε λεωφορειο πιο συγκεκριμενα γεγονος συνδεεται οπως φανηκε ληξη μαθηματων επαλ ευχαριστω'
B-HUR νοσοκομειο
B-HUR νοσοκομειου
B-HUR νοσοκομειου

Entities in 'καλησπερα σαςθα ηθελα καταγγειλω οδηγο λεωφορειου της κυκλικης διαδρομης πινακιδες yne ακολουθω καθημερινα τη γραμμη καθως κατευθυνομαι εργασια εχω γινει ματυρας ηδη δυο κακων συμβαντων συκεκριμενο ανευθυνο αγενη οδηγο τοσο προς μια κυρια προς προσωπο χτυπησαμε κανονικα κουμπια κανει σταση ωστοσο χτυπησε χαρακτηριστικος ηχος ειδοποιει οδηγοοταν φωναξα σταση απαντησεοτι κανει σταση π

## Use the saved model to recognize the sos comments and save a csv that contains the complaints that are considered important

In [ ]:
def create_sos_column(text, my_nlp):
    # print("\n", text)
    doc = my_nlp(text)
    for ent in doc.ents:
        # print(ent.label_, ent.text)
        if ent.label_ in ["B-LAW", "B-HUR"]:
          return 1
    return 0

new_df = data.copy()

output_dir = '/content/drive/My Drive/arximhdhs/ner/new_our_NER_model'
nlp2 = spacy.load(output_dir)

new_df['SOS'] = data['text'].apply(lambda x: create_sos_column(x, nlp2))

df_sos = new_df[new_df['SOS'] == 1]

def create_ent_column(text, my_nlp):
    doc = my_nlp(text)
    return doc.ents

final_df = df_sos.copy()

final_df['ents'] = df_sos['text'].apply(lambda x: create_ent_column(x, nlp2))

final_df = final_df.drop(['text'], axis=1)
final_df = final_df.reset_index(drop = True)
final_df.to_csv('/content/drive/My Drive/arximhdhs/ner/sos_comments_new.csv')
final_df

,init_text,SOS,ents
0,Αγαπητοί κύριοι καλησπέρα σας σήμερα 31/1 2022...,1,"((αποζημιωσετε),)"
1,"Παρακαλώ πολύ, σήμερα, Κυριακή, 30 /01 / 2022 ...",1,"((μηνυση),)"
2,Aπό χθες το πρωι δεν υπαρχει και δεν ξεκιναει ...,1,"((αποζημιωσει),)"
3,Πρέπει να μετά κινηθώ στη εργασία μου στο Λαϊκ...,1,"((νοσοκομειο),)"
4,"Το πρωί του Σαββάτου, έχοντας σχολασει απ' τη ...",1,"((χτυπαω),)"
...,...,...,...
759,"Καλημέρα, ονομάζομαι Δανάη Ορφανού Τσώτα. Επικ...",1,"((χτυπησα), (τραυματιστει))"
760,Τι θα γίνει ρε ΟΑΣΑ με τους οδηγούς σου που κα...,1,"((μηνυση),)"
761,Έσταζε όλη η οροφή από τα νερά της βροχής στον...,1,"((χτυπησεις),)"
762,ΑΠΑΡΑΔΕΚΤΟΣ ΟΔΗΓΟΣ ΤΟΥ ΥΠ ΑΡΘΜΟΝ ΧΕΚ 6168 ΣΤΗΝ...,1,"((δικαιοσυνη), (νομικες))"
